<a href="https://colab.research.google.com/github/tejb96/ml-pipelines/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4: Pipelines and Hyperparameter Tuning (32 total marks)
### Due: November 22 at 11:59pm

### Name: Tejpreet Bal

### In this assignment, you will be putting together everything you have learned so far. You will need to find your own dataset, do all the appropriate preprocessing, test different supervised learning models and evaluate the results. More details for each step can be found below.

### You will also be asked to describe the process by which you came up with the code. More details can be found below. Please cite any websites or AI tools that you used to help you with this assignment.

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score

## Step 1: Data Input (4 marks)

Import the dataset you will be using. You can download the dataset onto your computer and read it in using pandas, or download it directly from the website. Answer the questions below about the dataset you selected.

To find a dataset, you can use the resources listed in the notes. The dataset can be numerical, categorical, text-based or mixed. If you want help finding a particular dataset related to your interests, please email the instructor.

**You cannot use a dataset that was used for a previous assignment or in class**

In [ ]:
# Import dataset (1 mark)
df=pd.read_excel('probowl2022.xlsx')
df.head()



,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Succ%,Lng,Y/Tgt,R/G,Y/G,Fmb
0,1,Justin Jefferson*+,MIN,23,WR,17,17,184,128,0.696,1809,14.1,8,80,57.6,64,9.8,7.5,106.4,0
1,2,Tyreek Hill*+,MIA,28,WR,17,17,170,119,0.700,1710,14.4,7,77,56.5,64,10.1,7.0,100.6,1
2,3,Travis Kelce*+,KAN,33,TE,17,17,152,110,0.724,1338,12.2,12,78,63.2,52,8.8,6.5,78.7,1
3,4,Stefon Diggs*,BUF,29,WR,16,16,154,108,0.701,1429,13.2,11,74,64.3,53,9.3,6.8,89.3,1
4,5,Austin Ekeler,LAC,27,RB,17,17,127,107,0.843,722,6.7,5,36,48.0,23,5.7,6.3,42.5,5


In [ ]:
df.describe()


,Rk,Age,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Succ%,Lng,Y/Tgt,R/G,Y/G,Fmb
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,488.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000
mean,250.500000,25.920000,12.378000,5.650000,34.586000,23.210000,0.685114,254.076000,10.330533,1.500000,12.184000,47.814000,29.060000,6.728600,1.72560,18.663200,0.690000
std,144.481833,2.926214,4.704826,5.810251,37.679093,25.362184,0.203655,312.712469,5.405415,2.207203,15.332329,21.610532,18.939848,3.581292,1.62995,20.145802,1.206985
min,1.000000,21.000000,1.000000,0.000000,1.000000,0.000000,0.000000,-10.000000,-6.000000,0.000000,0.000000,0.000000,-6.000000,-6.000000,0.00000,-1.500000,0.000000
25%,125.750000,24.000000,9.000000,0.000000,6.000000,4.000000,0.590250,33.000000,7.000000,0.000000,1.000000,37.500000,15.000000,4.700000,0.50000,4.275000,0.000000
50%,250.500000,25.000000,14.000000,3.000000,20.000000,13.000000,0.692000,129.500000,9.900000,1.000000,6.000000,49.550000,26.000000,6.550000,1.10000,11.150000,0.000000
75%,375.250000,28.000000,16.000000,11.000000,52.000000,35.000000,0.800000,378.750000,12.725000,2.000000,18.000000,57.100000,41.000000,8.400000,2.60000,27.650000,1.000000
max,500.000000,45.000000,17.000000,17.000000,184.000000,128.000000,1.000000,1809.000000,42.000000,14.000000,80.000000,100.000000,98.000000,33.000000,8.30000,106.400000,8.000000


### Questions (3 marks)

1. (1 mark) What is the source of your dataset?
1. (1 mark) Why did you pick this particular dataset?
1. (1 mark) Was there anything challenging about finding a dataset that you wanted to use?

*ANSWER HERE*

1. I am using the NFL pro bowl 2022 dataset from:https://www.pro-football-reference.com/years/2023/receiving.htm?sr&utm_source=direct&utm_medium=Share&utm_campaign=ShareTool#receiving
2. I picked this dataset because I want to build a machine learning model that would be able to predict if a player is selected for the Pro Bowl based on the player stats.
3. It was not challenging to find the data.  



## Step 2: Data Processing (5 marks)

The next step is to process your data. Implement the following steps as needed.

In [ ]:
# Clean data (if needed)
df.isnull().sum()

# null_yr_rows = df[df['Y/R'].isnull()]
# print(null_yr_rows)
df['Y/R'].fillna(0, inplace=True)
df['Pos'] = df['Pos'].replace('LB/OLB', 'LB')
df['Pos'].fillna('LB', inplace=True)

# this resulted in inconsistent results everytime I hit run because I am filling with different values each time for position
# df.loc[df['Pos'].isnull(), 'Pos'] = random_positions = np.random.choice(df['Pos'].dropna().unique(), df['Pos'].isnull().sum())

# df.dropna(subset='Y/R',inplace=True)
# df.dropna(subset='Pos',inplace=True)
# print(df['Pos'].unique())

df.drop('Rk', axis=1, inplace=True)
df.drop('Tm', axis=1, inplace=True)
df.isnull().sum()

Player    0
Age       0
Pos       0
G         0
GS        0
Tgt       0
Rec       0
Ctch%     0
Yds       0
Y/R       0
TD        0
1D        0
Succ%     0
Lng       0
Y/Tgt     0
R/G       0
Y/G       0
Fmb       0
dtype: int64

In [ ]:
# Implement preprocessing steps. Remember to use ColumnTransformer if more than one preprocessing method is needed

df = pd.get_dummies(df, columns=['Pos'], drop_first=True) # encodes player position column
df['ProBowl'] = df['Player'].str.contains('*', regex=False).astype(int)
df.drop('Player', axis=1, inplace=True)
X = df.drop('ProBowl', axis=1)
y = df['ProBowl']
X.head()
# y.isnull().sum()
# y.sum()


,Age,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,...,R/G,Y/G,Fmb,Pos_FB,Pos_LB,Pos_QB,Pos_RB,Pos_T,Pos_TE,Pos_WR
0,23,17,17,184,128,0.696,1809,14.1,8,80,...,7.5,106.4,0,0,0,0,0,0,0,1
1,28,17,17,170,119,0.700,1710,14.4,7,77,...,7.0,100.6,1,0,0,0,0,0,0,1
2,33,17,17,152,110,0.724,1338,12.2,12,78,...,6.5,78.7,1,0,0,0,0,0,1,0
3,29,16,16,154,108,0.701,1429,13.2,11,74,...,6.8,89.3,1,0,0,0,0,0,0,1
4,27,17,17,127,107,0.843,722,6.7,5,36,...,6.3,42.5,5,0,0,0,1,0,0,0


### Questions (2 marks)

1. (1 mark) Were there any missing/null values in your dataset? If yes, how did you replace them and why? If no, describe how you would've replaced them and why.
2. (1 mark) What type of data do you have? What preprocessing methods would you have to apply based on your data types?

*ANSWER HERE*

1. There were null values for Y/R which is a ratio of Yds over Rec. The null value was due to both Rec and Yds being 0. I filled the missing values with 0 using fillna instead of the mean since it made more sense in this case. There were missing values for the positions column which I filled with random positions.
2. I have a mixed dataset containing numerical and categorical. I converted the categorical variable Pos to numerical using one-hot encoding. An asterisk at the end of the name is used to show that the player was selected for the Pro Bowl. I created a target column y which contain 1 for players with the * and 0 for without. The Rk (rank) and Tm (team) columns were dropped.

## Step 3: Implement Machine Learning Model (11 marks)

In this section, you will implement three different supervised learning models (one linear and two non-linear) of your choice. You will use a pipeline to help you decide which model and hyperparameters work best. It is up to you to select what models to use and what hyperparameters to test. You can use the class examples for guidance. You must print out the best model parameters and results after the grid search.

In [ ]:
# Implement pipeline and grid search here. Can add more code blocks if necessary
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Pipeline
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())
])

# Logistic Regression Pipeline
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=2500))
])

# SVM Pipeline
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])

# param grids

param_grid_rf = {
    'classifier__n_estimators': [50, 100, 150, 200, 250],
    'classifier__max_depth': [5, 10, 15, 20, 25]
}

param_grid_lr = {
    'classifier__C': [0.01, 0.1, 1],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__solver': ['liblinear', 'saga']
}

param_grid_svm = {
    'classifier__C': [0.1, 1, 10, 100],
    'classifier__kernel': ['linear', 'rbf']
}

# Define the scoring metrics you want to use
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score)
}

grid_search_rf = GridSearchCV(rf_pipeline, param_grid_rf, cv=5, scoring=scoring, refit='f1_score')
grid_search_lr = GridSearchCV(lr_pipeline, param_grid_lr, cv=5, scoring=scoring, refit='f1_score')
grid_search_svm = GridSearchCV(svm_pipeline, param_grid_svm, cv=5, scoring=scoring, refit='f1_score')

# Fit the models
grid_search_rf.fit(X_train, y_train)
grid_search_lr.fit(X_train, y_train)
grid_search_svm.fit(X_train, y_train)

# Get the best parameters based on F1
best_params_rf = grid_search_rf.best_params_
best_params_lr = grid_search_lr.best_params_
best_params_svm = grid_search_svm.best_params_

# Access the results for both scoring metrics
results_rf = grid_search_rf.cv_results_
results_lr = grid_search_lr.cv_results_
results_svm = grid_search_svm.cv_results_

# Print the results for accuracy and F1
print("Random Forest Results:")
print("Accuracy score:", results_rf['mean_test_accuracy'][results_rf['mean_test_f1_score'].argmax()])
print("F1 score:", max(results_rf['mean_test_f1_score']))
print("\nBest Parameters for Random Forest based on F1:", best_params_rf)

print("\nLogistic Regression Results:")
print("Accuracy scores:", results_lr['mean_test_accuracy'][results_lr['mean_test_f1_score'].argmax()])
print("F1 scores:", max(results_lr['mean_test_f1_score']))
print("\nBest Parameters for Logistic Regression based on F1:", best_params_lr)

print("\nSVM Results:")
print("Accuracy scores:", results_svm['mean_test_accuracy'][results_svm['mean_test_f1_score'].argmax()])
print("F1 scores:", max(results_svm['mean_test_f1_score']))
print("\nBest Parameters for SVM based on F1:", best_params_svm)

Random Forest Results:
Accuracy score: 0.9475
F1 score: 0.46761904761904766

Best Parameters for Random Forest based on F1: {'classifier__max_depth': 15, 'classifier__n_estimators': 50}

Logistic Regression Results:
Accuracy scores: 0.9324999999999999
F1 scores: 0.4371184371184372

Best Parameters for Logistic Regression based on F1: {'classifier__C': 0.01, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'}

SVM Results:
Accuracy scores: 0.9574999999999999
F1 scores: 0.5721212121212121

Best Parameters for SVM based on F1: {'classifier__C': 10, 'classifier__kernel': 'rbf'}


### Questions (5 marks)

1. (1 mark) Do you need regression or classification models for your dataset?
1. (2 marks) Which models did you select for testing and why?
1. (2 marks) Which model worked the best? Does this make sense based on the theory discussed in the course and the context of your dataset?

*ANSWER HERE*

1. I need classification models since the model will be predicting a categorical outcome, either the player is selected or not.
2. I chose logistic regression for the linear model because logistic regression fits well to binary classification problems. Random forest was chosen because it builds multipe decision trees and combines their results, which can result in better accuracy and prevent overfitting. SVM was chosen since it performs relatively well if there are any outliers in the dataset.
3. SVM model fit the dataset the best, with a F1 score of 0.57, Random Forest has F1 score of 0.45, and logistic regression has F1 score of 0.44. In class we discussed that SVM can fit really well to nonlinear and complex data and can separate data with a linear plane which resulted in the SVM fitting the best to this data.

## Step 4: Validate Model (6 marks)

Use the testing set to calculate the testing accuracy for the best model determined in Step 3.

In [ ]:
# Calculate testing accuracy (1 mark)
y_pred_svm = grid_search_svm.predict(X_test)
testing_accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Testing Accuracy:", testing_accuracy_svm)

f1=f1_score(y_test, y_pred_svm)
print("Test F1 Score:", f1)

SVM Testing Accuracy: 0.97
Test F1 Score: 0.5714285714285715



### Questions (5 marks)

1. (1 mark) Which accuracy metric did you choose?
1. (1 mark) How do these results compare to those in part 3? Did this model generalize well?
1. (3 marks) Based on your results and the context of your dataset, did the best model perform "well enough" to be used out in the real-world? Why or why not? Do you have any suggestions for how you could improve this analysis?

*ANSWER HERE*

1. I used F1 score and testing accuracy as the metrics. Since only 30 players are selected for pro bowl, F1 score is a good metric due to the imbalanced classes. I also did accuracy since there is no major downside to predicting false negatives.
2. This model shows consistency with results from part 3. F1 score was 0.57 for part 3 and remained the same, therefore model did generalize well. The accuracy in part 3 was 0.96, which is also about the same.
3. The model has an accuracy of 0.97, which is execellent. The F1 score is not that high which could mean that either precision or recall or both are low for this model. I dont think that this model is good enough currently to be used in the real world because of the low F1 score. I was able to improve the F1 score from 0.45 to 0.57 just by changing what I fill the empty values for the Pos column. I was filling it with random values and changed it to LB instead. To improve the model, i can revisit the feature pre-processing step again, also I can try different models as well.

## Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

*DESCRIBE YOUR PROCESS HERE*
1. I used some code from lab 6, such as for the encoding and also the lecture examples.
2. I attempted to complete he steps in order but ended up revisiting many of the steps multiple times.
3. I used AI to understand my results better. Initially, I was filling the NaN values in the Pos column with other randomly selected values from the same column, I was getting different results for the F1 score ranging between 0.45 to 0.50. For part 4 it remained consistent at 0.57, which seemed a bit odd. I ended up just filling it with LB which did give me more consistent results.
4. I used the labs and the lecture examples provided, which helped me to successfully complete this.

## Reflection (2 marks)
Include a sentence or two about:
- what you liked or disliked,
- found interesting, confusing, challenging, motivating
while working on this assignment.


*ADD YOUR THOUGHTS HERE*

I liked that we got to chose our own data.

Something I found interesting was that, accuracy is not always important. My model had a high accuracy, but if it simply predicts that no one will be selected for the Pro Bowl, it would still have a high accuracy; considering the imbalance of only 30 players being selected out of approximately 500 in the dataset. The F1 score is much more important to determine if the model is good enough, since it accounts for precision and recall.